In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
from keras.utils.visualize_util import plot
from keras.models import model_from_json



%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
import time

nb_epoch = 5

In [ ]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_train = X_train.shape[0]
num_test = X_test.shapeOn[0]
im_width  = X_train.shape[1]
im_height = X_train.shape[2]
print('shape before')
print(X_train.shape)
X_train = X_train.reshape(num_train,1, im_width,im_height)
X_test = X_test.reshape(num_test,1, im_width,im_height)

print('shape after')
print(X_train.shape)

In [ ]:
#Remove numbers 5 through 9
newX = []
newy = []
newXtest = []
newYtest = []
for i in range(len(y_train)):
    if y_train[i] < 5:
        newX.append(X_train[i])
        newy.append(y_train[i])
for i in range(len(y_test)):
    if y_test[i] < 5: 
        newXtest.append(X_test[i])
        newYtest.append(y_test[i])
X_train = np.array(newX)
y_train = np.array(newy)
X_test = np.array(newXtest)
y_test = np.array(newYtest)
        

In [ ]:
# change type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalize the range
print('maximum of X_train:',np.max(X_train[:]))
X_train /= 255.0;
X_test /= 255.0;
print('maximum of X_train:',np.max(X_train[:]))

# convert class vectors to binary class matrices (one hot representation)
nb_classes = np.unique(y_train).size
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:

# construct the network
model = Sequential()
model.add(Convolution2D(64, 10, 10, border_mode='same', input_shape=(1,im_width,im_height)))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(Convolution2D(16, 2, 2, border_mode='same', input_shape=(1,im_width,im_height)))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(MaxPooling2D(pool_size=(2,2), strides=None, border_mode='valid'))
model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
batch_size = 128
start = time.time()
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
end = time.time()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)

print ('Test score:   ', score[0])
print( 'Test accuracy:', score[1])
print( 'Time elapsed: ',(end - start), "seconds")

In [ ]:
print (model)

In [ ]:
plot(model, to_file='model.png')

In [ ]:
img = mpimg.imread('model.png')
plt.imshow(img)

In [ ]:
#Save the weights of the model
model.save_weights('modelWeights.h5',overwrite=True)

#Save the architecture of the model
json_string_modelArchitecture = model.to_json()
open('my_model_architecture.json', 'w').write(json_string_modelArchitecture)


In [ ]:
if 'model' in dir():
    print ("deleted model")
    del model
else:
    print ("deleted already")

In [ ]:
#Load the old architecture
model = model_from_json(open('my_model_architecture.json').read())
model.load_weights('modelWeights.h5')

In [ ]:
#Show the model to make sure the load in worked
model.summary()

In [ ]:
#pop the classification layers
model.layers.pop() #remove softmax activation
model.layers.pop() #remove classification output layer
model.layers.pop() #remove dropout
model.layers.pop() #remove Relu Activation
model.layers.pop() #remove fully connected layer
model.summary()

In [ ]:
for i in range(len(model.layers)):
    model.layers[i].train=False
    print ("Freezing ", model.layers[i].name)

In [ ]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()
num_train = X_train.shape[0]
num_test = X_test.shape[0]
im_width  = X_train.shape[1]
im_height = X_train.shape[2]
print('shape before')
print(X_train.shape)
X_train = X_train.reshape(num_train,1, im_width,im_height)
X_test = X_test.reshape(num_test,1, im_width,im_height)

print('shape after')
print(X_train.shape)

In [ ]:
# change type to float
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalize the range
print('maximum of X_train:',np.max(X_train[:]))
X_train /= 255.0;
X_test /= 255.0;
print('maximum of X_train:',np.max(X_train[:]))

# convert class vectors to binary class matrices (one hot representation)
nb_classes = np.unique(y_train).size
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

In [ ]:
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.23))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))
model.summary()

In [ ]:
#Compile the new model
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [ ]:
batch_size = 128

start = time.time()
history = model.fit(X_train, Y_train,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=1, validation_data=(X_test, Y_test))
end = time.time()

In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)

print ('Test score:   ', score[0])
print( 'Test accuracy:', score[1])
print( 'Time elapsed: ',(end - start), "seconds")